# Where is Bill Project

In [128]:
# Definition of functions

def getrecentbill(query):
    headers={ "X-API-Key": "OL9rTNU3pOCB5FkqOWJH7hMAdmCZLS3GJU5NpLuV" }
    response = requests.get(url, headers=headers)
    bill = response.json()
    bill_data = []
    for i in bill['results'][0]['bills']:
        bills = {}
        bill_data.append(bills)
        bills['bill_id'] = i["bill_id"]
        bills['bill_title'] = i['title']
        bills['latest_major_action_data'] = i['latest_major_action_date']
        bills['latest_major_action'] = i['latest_major_action']
    return bill_data

In [129]:
# Main program

import json
import requests


query = input("Please enter a subject: ")
url = "https://api.propublica.org/congress/v1/bills/search.json?query=" + query
bill_data = getrecentbill(subject)
print(bill_data)


Please enter a subject: tax
[{'bill_id': 'hr4470-115', 'bill_title': 'To amend the Internal Revenue Code of 1986 to provide a payroll tax exemption for hiring long-term unemployed individuals.', 'latest_major_action_data': '2017-11-28', 'latest_major_action': 'Referred to the House Committee on Ways and Means.'}, {'bill_id': 'hr4460-115', 'bill_title': 'To improve the provision of disaster and mitigation assistance to eligible individuals and households and to eligible State, local, Tribal, and territorial governments and certain private nonprofit organizations, and for other purposes.', 'latest_major_action_data': '2017-11-28', 'latest_major_action': 'Referred to the Committee on Transportation and Infrastructure, and in addition to the Committee on Financial Services, for a period to be subsequently determined by the Speaker, in each case for consideration of such provisions as fall within the jurisdiction of the committee concerned.'}, {'bill_id': 'hr1-115', 'bill_title': 'To provid